# fmdtools-AdaStress integration


**NOTE: This notebook may not run correctly, see ticket RAD-224**

This goal of this notebook is to illustrate running [Adaptive Stress Testing](https://www.nasa.gov/content/tech/rse/research/adastress) on fmdtools model using the AdaStress package.

For this setup to work, first install:
- AdaStress (and related packages) in Julia
- fmdtools (and related packages) in Python
- PyCall in Julia
- PyJulia in Python

In [1]:
using Plots
using AdaStress

### Python setup:

First, we get the fmdtools model from its respective folder

In [2]:
using PyCall
#ENV["MPLBACKEND"]="qt5agg"
using PyPlot
py"""
import sys, os
sys.path.insert(0,os.curdir)
sys.path.insert(1,os.path.join('..'))
"""

In [3]:
#pygui_start(:qt)
#PyCall.fixqtpath()

Next, we get the module--in this case we will be using the stochastic pump model in `/example_pump/pump_stochastic`

In [4]:
pump_module = pyimport("pump_stochastic")
pump_model = pump_module.Pump()

To get the propagation/analysis methods, we can further import the fmdtools packages...

In [5]:
prop = pyimport("sim.propagate")
an = pyimport("analyze")

Example simulation

In [6]:
endresult, mdlhist = prop.nominal(pump_model, run_stochastic=true);

NOTE: in order for `an.plot.mdlhists` to work correctly, make sure to use the `Using PyCall` command at the beginning.

In [7]:
fig, axs = an.plot.hist(mdlhist, "fxns.move_water.s.eff", 
                                 "fxns.move_water.s.total_flow", 
                                 "flows.wat_2.s.flowrate", 
                                 "flows.wat_2.s.pressure")

## AdaStress Setup

Interfacing AdaStress with fmdtools models is enabled by the `DynamicProblem` class in `sim.search` (though this can also be done manually). See below. 

In [8]:
search = pyimport("sim.search")
DynamicProblem = search.DynamicInterface

In [9]:
prob = DynamicProblem(pump_model)

Defining the simulation is quite simple with the DynamicProblem class. Note the possible options from this class

In [10]:
?DynamicProblem.__init__

In [11]:
Base.@kwdef mutable struct PumpSim <: AdaStress.BlackBox
    sim::PyObject = DynamicProblem(pump_model)
end

In [12]:
function initialize!(sim::PumpSim)
    sim.sim = DynamicProblem(pump_model)
end

Updating is further enabled by DynamicProblem.update

In [13]:
?DynamicProblem.update

In [14]:
function update!(sim::PumpSim)
    seed = rand(0:1_000_000_000)
    returns = sim.sim.update(seed=seed)
    return log(returns["pdf"]) # note: needs to return a probability from somewhere - needs to be a log pdf?
end

Model end-states can be interfaced with using `DynamicProblem.check_sim_end`, which additionally accomodates external conditions.

In [15]:
function isterminal!(sim::PumpSim)
    has_faults = sim.sim.mdl.fxns["move_water"].m.any_faults() # our external condition
    terminal = sim.sim.check_sim_end(external_condition=has_faults)
    return terminal
end

In [16]:
function isevent!(sim::PumpSim)
    faults = sim.sim.mdl.fxns["move_water"].m.any_faults()
    return faults
end

Finally, the below shows the full setup:

In [17]:
AdaStress.reset!(sim::PumpSim) = initialize!(sim)
AdaStress.step!(sim::PumpSim) = update!(sim)
AdaStress.isterminal(sim::PumpSim) = isterminal!(sim)
AdaStress.isevent(sim::PumpSim) = isevent!(sim)     # difficult: get pressure > 15 ten times
# AdaStress.isevent(sim::PumpSim) = 15.0-sim.mdl.flows["Wat_1"].pressure <= 0 # - easy: just get pressure > 15 (once!)
#AdaStress.distance(sim::PumpSim) = 15.0-sim.mdl.flows["Wat_1"].pressure
AdaStress.distance(sim::PumpSim) = float(sum(sim.sim.hist.flows.wat_1.s.pressure.<=15))

### Running AdaStress Methods

Monte Carlo Search

In [18]:
mcs = AdaStress.Solvers.MCS(num_iterations=10_00)

In [19]:
mdp = AdaStress.ASTMDP(PumpSim())
mdp.reward.event_bonus = 200.0

In [20]:
using Random

In [21]:
Random.seed!(0)
sol = mcs(() -> mdp)

In [45]:
i=0
mdlhists = Dict()
for (result, r) in sol
    AdaStress.replay!(mdp, result)
    mdlhists[string("scen", i)] = mdp.sim.sim.hist
    i+=1
end
mdlhists = an.result.History(mdlhists)
mdlhists = mdlhists.flatten()

In [52]:
an.plot.hist(mdlhists, "fxns.move_water.s.eff", 
                        "fxns.move_water.s.total_flow", 
                        "flows.wat_2.s.flowrate", 
                        "flows.wat_2.s.pressure", comp_groups=Dict())

### MCTS

Monte Carlo Tree Search

In [28]:
mcts = AdaStress.Solvers.MCTS(num_iterations=1000)

In [29]:
Random.seed!(0)
@time sol = mcts(() -> mdp)

In [30]:
i=0
mdlhists2 = Dict()
for (result, r) in sol
    AdaStress.replay!(mdp, result)
    mdlhists2[i] = mdp.sim.sim.log
    i+=1
end

In [31]:
an.plot.mdlhists(mdlhists2, fxnflowvals=fxnflows2, aggregation="percentile")

In [32]:
fxnflows = Dict("MoveWater"=>["pressure","faults"], "Wat_2"=>["flowrate","pressure"])
an.plot.mdlhists(mdlhists2, fxnflowvals=fxnflows, legend_loc=1)

As we can see, MCTS finds more breakage faults (8 instead of 3), as a result of being a more informed search method.

## Grey Box Methods

As opposed to black box methods, which share no information about the system state with the agent, grey box methods let the agent sample the random distributions and pass disturbances to the model directly, rather than just passing seeds.

Implementation is based off of: https://github.com/NASA-SW-VnV/AdaStress.jl/blob/master/examples/walk2d/walk2d.ipynb

In [33]:
using AdaStress
import AdaStress.GrayBox
using Distributions
using Random

### Pump

To use the gray box method, we have to first define which variables in the model to sample/update. fmdtools models have two types of random states:
- general random states, which are updated in the behavioral method (whose distributions thus cannot be accessed without looking at the code)
- auto-updating random states, which are updated from a given distribution defined with auto_update=(dist,params). These distributions can be accessed directly from the model.

Both can be queried in AdaStress, but using different methods. In general, however, using a Gray Box method means that *all* stochastic behavior (desired by the analysis) should be represented in AdaStress/Julia, since integration relies on running the model as deterministic.

Below are the random states of the model:

In [34]:
rand_states = pump_model.get_rand_states()

As well as the auto-updating random states:

In [35]:
rand_states = pump_model.get_rand_states(auto_update_only=true)

Because of the different methods that julia/AdaStress uses to sample random distributions, these distributions need to be re-implemented in julia. In this example we do this for the eff parameter in MoveWater.

Below we create a `Pump` struct, and define random disturbances which will affect the model.

In [36]:
#define distributions for the model
#Base.@kwdef mutable struct Init_Rand_Params
#    eff::Distribution = Normal(1.0, 0.2)
#end
#Base.@kwdef mutable struct Pump
#    t::Float64=0.0
#    t_max::Float64=last(pump_model.times)
#    t_ind::Int=0
#    mdl::PyObject = pump_model
#end
#function initialize(pump::Pump, init::Init_Rand_Params)
#    pump.mdl = pump.mdl.__class__()
#    pump.mdl.fxns["MoveWater"].eff = rand(init.eff)
#end

#function update(pump::Pump, eff::Float64)
#    pump.mdl.fxns["MoveWater"].eff = eff
#end

Disturbance and failure models

In [37]:
#Base.@kwdef mutable struct Disturbance
#    eff::Distribution = Normal(1.0, 0.2)
#end

System under test: okay to remove, often refers to policy that we are trying to test within the system dynamics

In [38]:
#mutable struct SystemUnderTest
#    eff::Float64
#end
#update(pump::Pump, sut::SystemUnderTest) = update(pump, sut.eff)

### Simulator
The simulator is roughly the same as the simulation for the black box model except it has an `environment` which samples stochastic variables and then changes parameters in the model. Note that in this case `run_stochastic` is set to `false`, since the variables are being sampled in AdaStress itself (rather than the model).

In [39]:
Base.@kwdef mutable struct Simulator <: AdaStress.GrayBox
    sim::PyObject = DynamicProblem(pump_model)
    env::AdaStress.Environment = AdaStress.Environment()
end

function initialize(sim::Simulator)
    sim.env[:eff] = Normal(1.0, 0.2) #creating  
    sim.sim=DynamicProblem(pump_model, run_stochastic=false)
    return
end

function update!(sim::Simulator, value::AdaStress.EnvironmentValue)
    #sim.mdl.set_values("MoveWater.eff", value[:eff])
    disturbances=Dict("MoveWater.eff"=>value[:eff])
    sim.sim.update(disturbances=disturbances)
    return
end

The isterminal and isevent functions will be the same as in the black box model.

In [40]:
function isterminal!(sim::Simulator)
    has_faults = sim.sim.mdl.fxns["MoveWater"].any_faults() # our external condition
    terminal = sim.sim.check_sim_end(external_condition=has_faults)
    return terminal
end
function isevent!(sim::Simulator)
    faults = sim.sim.mdl.fxns["MoveWater"].any_faults() 
    return faults
end

The interface to adastress is set up below. Of note is the observation interface, which observes `eff` (which is also the action in this simple setup)

In [41]:
Interface.reset!(sim::Simulator) = initialize(sim)

Interface.environment(sim::Simulator) = sim.env

Interface.observe(sim::Simulator) = vcat(sim.sim.mdl.flows["Wat_2"].pressure, sim.sim.t / sim.sim.t_max)

Interface.step!(sim::Simulator, value::AdaStress.EnvironmentValue) = update!(sim, value)

Interface.isterminal(sim::Simulator) = isterminal!(sim)

Interface.isevent(sim::Simulator) = isevent!(sim)

Interface.distance(sim::Simulator) = float(sum([(16-max(min(i,16), 0))^2 for i in sim.sim.log["flows"]["Wat_1"]["pressure"]]))

In [42]:
function mdp_env(; kwargs...)
    mdp = Interface.ASTMDP(Simulator(; kwargs...))
    mdp.rewaan.event_bonus = 2000.0
    return mdp
end

Now using the gray box mdp solver:

In [43]:
AdaStress.enable("SoftActorCritic")
using AdaStress.SoftActorCritic

In [44]:
Random.seed!(0)
sac = SAC(;
    obs_dim=2, 
    act_dim=1,
    gamma=0.95,
    act_mins=-1*ones(1),    #number of standard deviations
    act_maxs=10*ones(1),
    hidden_sizes=[30,30,30],
    num_q=2,
    max_buffer_size=1000000,
    batch_size=128,
    epochs=100,
    update_after=0,
    update_every=1,
    steps_per_epoch=54,
    start_steps=100,
    max_ep_len=54,
    num_test_episodes=100,
    displays=[(:fails, mdp -> mdp.sim.sim.mdl.fxns["MoveWater"].has_fault("mech_break") )],
    use_gpu=false
)

SoftActorCritic.ProgressMeter.ijulia_behavior(:clear)
ac, info = sac(mdp_env)

In [45]:
using PyPlot

In [46]:
plt.plot(info["fails"])

In [47]:
plt.plot(info["score"])

In [48]:
ac

In [49]:
i=0
mdlhists3 = Dict()
mdp = mdp_env()
for i in 1:100
    a= AdaStress.replay!(mdp, ac)
    mdlhists3[i] = mdp.sim.sim.log
    i+=1
end

In [50]:
a= AdaStress.replay!(mdp, ac)

In [51]:
info

In [52]:
fxnflows = Dict("MoveWater"=>["eff","total_flow"], "Wat_2"=>["flowrate","pressure"])
an.plot.mdlhists(mdlhists3, fxnflowvals=fxnflows, legend_loc=0)

In [53]:
fxnflows = Dict("MoveWater"=>["pressure","faults"], "Wat_2"=>["flowrate","pressure"])
an.plot.mdlhists(mdlhists3, fxnflowvals=fxnflows, legend_loc=0)